In [187]:
from collections import OrderedDict
from urbansim_templates import modelmanager as mm
from urbansim_templates.models import LargeMultinomialLogitStep
from urbansim_templates.models import SmallMultinomialLogitStep
import orca
import os; os.chdir('../')
import warnings; warnings.simplefilter('ignore')

import pandas as pd
# import pandana as pdna
import time
import numpy as np
import matplotlib.pyplot as plt
import os
from functools import reduce

# import matplotlib
# matplotlib.style.use('ggplot')

%matplotlib inline

pd.options.display.max_columns = 80

In [303]:
trips = pd.read_csv('/home/emma/ual_model_workspace/fall-2018-models/notebooks-emma/HWtrips_101418.csv')

trips.head()

,Unnamed: 0,HHPER,HHPERTRIP,origin,origin_dwell,origin_ST,origin_ET,trip_ET,TT,Ozone,Dzone,Oxcord,Oycord,Dxcord,Dycord,MODE
0,0,10352742,1.035274e+09,home,13.583333,19.166667,8.75,9.166667,0.416667,424.0,700.0,-122.017274,37.388281,-121.664242,37.151629,drive_alone
1,1,10352742,1.035274e+09,work,7.583333,9.166667,16.75,19.166667,2.416667,700.0,424.0,-121.664242,37.151629,-122.017274,37.388281,drive_alone
2,2,10414944,1.041494e+09,home,14.500000,18.000000,8.50,8.750000,0.250000,1095.0,1152.0,-122.091521,38.010760,-122.118103,37.885783,drive_alone
3,3,10414944,1.041494e+09,work,8.750000,8.750000,17.50,18.000000,0.500000,1152.0,1095.0,-122.118103,37.885783,-122.091521,38.010760,drive_alone
4,4,10417661,1.041766e+09,home,13.250000,18.750000,8.00,9.000000,1.000000,241.0,69.0,-122.414811,37.610585,-122.447100,37.779020,drive_alone


In [304]:
TOD_list = ['EA','AM','MD','PM','EV']
trips['TOD'] = (
    ((trips.origin_ET.between(3,6,inclusive = False)) | (trips.origin_ET==3))*1 +
    ((trips.origin_ET.between(6,9,inclusive = False)) | (trips.origin_ET==6))*2 +
    ((trips.origin_ET.between(9,15.5,inclusive = False)) | (trips.origin_ET==9))*3 +
    ((trips.origin_ET.between(15.5,18.5,inclusive = False)) | (trips.origin_ET==15.5))*4 +
    ((trips.origin_ET>=18.5))*5 +
    ((trips.origin_ET.between(0,3,inclusive = False)) | (trips.origin_ET==0))*5
)
TOD_dict = dict(zip(range(1,6),TOD_list))
trips['TOD'] = trips['TOD'].replace(TOD_dict)

TOD_dict = dict(zip(['EA', 'AM', 'MD', 'PM', 'EV'],range(1,6)))
trips['TOD'] = trips['TOD'].replace(TOD_dict)

trips['TOD'] = trips['TOD'] - 1

In [275]:
person = pd.read_csv('/home/data/CHTS_csv_format/data/Deliv_PER.csv')

person.head()

,SAMPN,PERNO,RELAT,GEND,AGE,AGEB,HISP,RACE1,RACE2,RACE3,RACE4,O_RACE,NTVTY,CNTRY,LIC,USER,TRANS,TPTYP1,TPTYP2,TPTYP3,TPTYP4,TPTYP5,TPTYP6,TPTYP7,O_TPTYP,CLIP1,CLIP2,CLIP3,COMP,MET,PASSTL,FLEX,EMPLY,WKSTAT,O_WKSTAT,JOBS,WLOC,WNAME,WCITY,WSTAT,...,HVLOG,PTRIPS,TOLLF,TOLLR1,TOLLR2,TOLLR3,TOLLR4,TOLLR5,TOLLR6,TOLLR7,TOLLR8,TOLLR9,TOLLR10,TOLLB1,TOLLB2,TOLLB3,TOLLB4,TOLLB5,TOLLB6,TOLLB7,TOLLB8,TOLLB9,TOLLB10,HOVL,NOGOWHY,NOGOWHY_O,InComplete,Moto_trip,WCTFIP,WTRACT,SCTFIP,STRACT,WPrimaryCity,WSTFIP,W2PrimaryCity,W2STFIP,SPrimaryCity,SSTFIP,PERWGT,EXPPERWGT
0,1031985,1,1,1,74,NaN,2,1.0,NaN,NaN,NaN,NaN,1,NaN,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.052086,17.647568
1,1031985,2,2,2,73,NaN,2,1.0,NaN,NaN,NaN,NaN,1,NaN,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,...,1.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.052086,17.647568
2,1032036,1,1,1,46,NaN,2,1.0,NaN,NaN,NaN,NaN,1,NaN,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,1.0,NaN,NaN,1.0,1.0,HIDDEN,SAN DIEGO,CA,...,NaN,5.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,73.0,17032.0,NaN,NaN,SAN DIEGO,6.0,NaN,NaN,NaN,NaN,1.223974,414.701494
3,1032036,2,2,2,47,NaN,2,1.0,97.0,NaN,NaN,MULTI-RACIAL,1,NaN,1.0,1.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,2.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,18.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.863473,292.558373
4,1032036,3,3,1,15,NaN,2,1.0,97.0,NaN,NaN,MULTI-RACIAL,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,4.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,NaN,1.0,NaN,NaN,73.0,17030.0,NaN,NaN,NaN,NaN,SAN DIEGO,6.0,0.941412,318.965100


In [32]:
# person1 = person.loc[person['RACE1'] == 1.0]

In [45]:
# person['RACE'] = person['RACE1']

In [46]:
# person.loc[person.HISP == 1.0, 'RACE'] = 6.0

In [276]:
# person['RACE'].value_counts()

In [277]:
# person['RACE1'].value_counts()

In [278]:
# print (person['RACE1'].count())
# print (person['SAMPN'].count())

In [279]:
# person1['RACE2'].value_counts()

In [35]:
# person1['RACE3'].value_counts()

In [36]:
# person1['RACE4'].value_counts()

In [37]:
# person['O_RACE'].count()

In [38]:
# person['SAMPN'].count()

In [39]:
# person['O_RACE'].value_counts()

In [281]:
person = person[['SAMPN','PERNO','GEND','AGE','HISP','RACE1','RACE2','RACE3','RACE4','O_RACE','NTVTY','LIC','JOBS','HOURS','EDUCA','WSCHED','DISAB']]

# person = person[['SAMPN','PERNO','GEND','AGE','RACE','NTVTY','LIC','JOBS','HOURS','EDUCA','WSCHED','DISAB']]

In [283]:
person['HHPER'] = person['SAMPN'].map(str) + person['PERNO'].map(str)

person['female'] = person['GEND'] - 1

person['HISP'] = np.where(person['HISP'] == 2, 0, person['HISP'])
person['white'] = np.where((person['RACE1'].isin([1.0]) | person['RACE2'].isin([1.0]) | person['RACE3'].isin([1.0]) | person['RACE4'].isin([1.0])),1,0)
person['black'] = np.where((person['RACE1'].isin([2.0]) | person['RACE2'].isin([2.0]) | person['RACE3'].isin([2.0]) | person['RACE4'].isin([2.0])),1,0)
person['native'] = np.where((person['RACE1'].isin([3.0]) | person['RACE2'].isin([3.0]) | person['RACE3'].isin([3.0]) | person['RACE4'].isin([3.0])),1,0)
person['asian'] = np.where((person['RACE1'].isin([4.0]) | person['RACE2'].isin([4.0]) | person['RACE3'].isin([4.0]) | person['RACE4'].isin([4.0])),1,0)
person['PI'] = np.where((person['RACE1'].isin([5.0]) | person['RACE2'].isin([5.0]) | person['RACE3'].isin([5.0]) | person['RACE4'].isin([5.0])),1,0)

person['immigrant'] = person['NTVTY'] - 1
person['nolic'] = person['LIC'] - 1

person['DISAB'] = np.where(person['DISAB'] == 2, 0, person['HISP'])

person.head()

,SAMPN,PERNO,GEND,AGE,HISP,RACE1,RACE2,RACE3,RACE4,O_RACE,NTVTY,LIC,JOBS,HOURS,EDUCA,WSCHED,DISAB,female,white,black,native,asian,PI,immigrant,nolic,HHPER
0,1031985,1,1,74,0,1.0,NaN,NaN,NaN,NaN,1,1.0,NaN,NaN,6,NaN,0,0,1,0,0,0,0,0,0.0,10319851
1,1031985,2,2,73,0,1.0,NaN,NaN,NaN,NaN,1,1.0,NaN,NaN,6,NaN,0,1,1,0,0,0,0,0,0.0,10319852
2,1032036,1,1,46,0,1.0,NaN,NaN,NaN,NaN,1,1.0,1.0,40.0,6,2.0,0,0,1,0,0,0,0,0,0.0,10320361
3,1032036,2,2,47,0,1.0,97.0,NaN,NaN,MULTI-RACIAL,1,1.0,NaN,NaN,6,NaN,0,1,1,0,0,0,0,0,0.0,10320362
4,1032036,3,1,15,0,1.0,97.0,NaN,NaN,MULTI-RACIAL,1,NaN,NaN,NaN,1,NaN,0,0,1,0,0,0,0,0,NaN,10320363


In [285]:
hh_df = pd.read_csv('/home/data/CHTS_csv_format/data/Deliv_HH.csv')[
  ['SAMPN','HHVEH','HHBIC','OWN','INCOM','HHSIZ']]

hh_df['rent'] = hh_df['OWN'] - 1

hh_df.head()

,SAMPN,HHVEH,HHBIC,OWN,INCOM,HHSIZ,rent
0,1031985,2,2,1,3,2,0
1,1032036,1,4,1,7,5,0
2,1032053,2,2,2,2,6,1
3,1032425,2,3,1,7,2,0
4,1032558,0,0,2,1,1,1


In [286]:
demo = person.merge(hh_df,on = 'SAMPN',how = 'left')

demo.head()

,SAMPN,PERNO,GEND,AGE,HISP,RACE1,RACE2,RACE3,RACE4,O_RACE,NTVTY,LIC,JOBS,HOURS,EDUCA,WSCHED,DISAB,female,white,black,native,asian,PI,immigrant,nolic,HHPER,HHVEH,HHBIC,OWN,INCOM,HHSIZ,rent
0,1031985,1,1,74,0,1.0,NaN,NaN,NaN,NaN,1,1.0,NaN,NaN,6,NaN,0,0,1,0,0,0,0,0,0.0,10319851,2,2,1,3,2,0
1,1031985,2,2,73,0,1.0,NaN,NaN,NaN,NaN,1,1.0,NaN,NaN,6,NaN,0,1,1,0,0,0,0,0,0.0,10319852,2,2,1,3,2,0
2,1032036,1,1,46,0,1.0,NaN,NaN,NaN,NaN,1,1.0,1.0,40.0,6,2.0,0,0,1,0,0,0,0,0,0.0,10320361,1,4,1,7,5,0
3,1032036,2,2,47,0,1.0,97.0,NaN,NaN,MULTI-RACIAL,1,1.0,NaN,NaN,6,NaN,0,1,1,0,0,0,0,0,0.0,10320362,1,4,1,7,5,0
4,1032036,3,1,15,0,1.0,97.0,NaN,NaN,MULTI-RACIAL,1,NaN,NaN,NaN,1,NaN,0,0,1,0,0,0,0,0,NaN,10320363,1,4,1,7,5,0


In [287]:
demo = demo[['SAMPN','PERNO','HHPER','female','AGE','HISP','white','black','native','asian','PI','immigrant','nolic','JOBS','HOURS','EDUCA','WSCHED','DISAB','HHVEH','HHBIC','rent','INCOM','HHSIZ']]

In [289]:
na_dict = {
    'female':[8],
    'AGE':[998,999],
#     'RACE':[97,98,99],
    'immigrant':[7,8],
    'nolic':[7,8],
    'JOBS':[98,99],
    'HOURS':[998,999],
    'EDUCA':[7,8,9],
    'WSCHED':[8,9],
    'DISAB':[8,9],
    'HHVEH':[98,99],
    'HHBIC':[98,99],
#     'VEHNEW':[8,9],
    'rent':[6,7,8],
    'INCOM':[98,99],
    'HHSIZ':[98,99]
}

for col in na_dict:
    for vals in na_dict[col]:
        demo[col] = demo[col].replace(vals,np.nan)
        
# demo.dropna(inplace = True)

In [290]:
demo.isnull().sum(axis = 0)

SAMPN            0
PERNO            0
HHPER            0
female         311
AGE           3924
HISP             0
white            0
black            0
native           0
asian            0
PI               0
immigrant      325
nolic        19666
JOBS         57427
HOURS        59603
EDUCA         2161
WSCHED       58106
DISAB         1154
HHVEH            0
HHBIC          148
rent           339
INCOM         8839
HHSIZ            0
dtype: int64

In [291]:
demo1 = demo.copy()

demo1.dropna(inplace = True)

In [292]:
print (len(person.index))
print (len(hh_df.index))
print (len(demo.index))
print (len(demo1.index))

109113
42431
109113
43487


In [305]:
trips['HHPER'] = trips['HHPER'].map(str)

trips0 = pd.merge(trips, demo, on='HHPER')

trips1 = pd.merge(trips, demo1, on='HHPER')

print (len(trips.index))
print (len(trips0.index))
print (len(trips1.index))

11595
11595
9895


In [296]:
trips0.isnull().sum(axis = 0)

Unnamed: 0        0
HHPER             0
HHPERTRIP         0
origin            0
origin_dwell    208
origin_ST       207
origin_ET         0
trip_ET          59
TT               59
Ozone             0
Dzone             0
Oxcord            0
Oycord            0
Dxcord            0
Dycord            0
MODE              0
SAMPN             0
PERNO             0
female           32
AGE             493
HISP              0
white             0
black             0
native            0
asian             0
PI                0
immigrant        25
nolic             4
JOBS             25
HOURS           276
EDUCA           158
WSCHED          117
DISAB           118
HHVEH             0
HHBIC             8
rent             18
INCOM           947
HHSIZ             0
dtype: int64

In [97]:
# x = trips0.loc[(trips0['VEHNEW'].isnull()) & (trips0['INCOM'].isnull())]

# len(x.index)

189

In [172]:
# trips = trips0.drop(columns=['VEHNEW'])

In [173]:
# trips.dropna(inplace = True)

# len(trips.index)

9496

In [67]:
# trips['HHPER'] = trips['HHPER'].map(str)

# trips = pd.merge(trips, demo, on='HHPER')

# trips.head()

In [119]:
skim_dir = '/home/data/fall_2018/mtc_skims'
distance_EA = pd.read_csv(skim_dir + f'/DistanceSkimsDatabaseEA.csv').replace(-999, 9999)
distance_AM = pd.read_csv(skim_dir + f'/DistanceSkimsDatabaseAM.csv').replace(-999, 9999)
distance_MD = pd.read_csv(skim_dir + f'/DistanceSkimsDatabaseMD.csv').replace(-999, 9999)
distance_PM = pd.read_csv(skim_dir + f'/DistanceSkimsDatabasePM.csv').replace(-999, 9999)
distance_EV = pd.read_csv(skim_dir + f'/DistanceSkimsDatabaseEV.csv').replace(-999, 9999)

cost_EA = pd.read_csv(skim_dir + f'/CostSkimsDatabaseEA.csv').replace(-999, 9999)
cost_AM = pd.read_csv(skim_dir + f'/CostSkimsDatabaseAM.csv').replace(-999, 9999)
cost_MD = pd.read_csv(skim_dir + f'/CostSkimsDatabaseMD.csv').replace(-999, 9999)
cost_PM = pd.read_csv(skim_dir + f'/CostSkimsDatabasePM.csv').replace(-999, 9999)
cost_EV = pd.read_csv(skim_dir + f'/CostSkimsDatabaseEV.csv').replace(-999, 9999)

Time_EA = pd.read_csv(skim_dir + f'/TimeSkimsDatabaseEA.csv').replace(-999, 9999)
Time_AM = pd.read_csv(skim_dir + f'/TimeSkimsDatabaseAM.csv').replace(-999, 9999)
Time_MD = pd.read_csv(skim_dir + f'/TimeSkimsDatabaseMD.csv').replace(-999, 9999)
Time_PM = pd.read_csv(skim_dir + f'/TimeSkimsDatabasePM.csv').replace(-999, 9999)
Time_EV = pd.read_csv(skim_dir + f'/TimeSkimsDatabaseEV.csv').replace(-999, 9999)

In [132]:
distance_EA = distance_EA.rename(
        columns = dict(zip([i for i in distance_EA.columns if i not in ['orig','dest']],
         [i+ f'_distance_EA' for i in distance_EA.columns if i not in ['orig','dest']])))

# distance_EA = distance_EA.add_suffix('_distance_EA')

# distance_EA.columns = [str(col) + '_x' for col in distance_EA.columns if col not in ['orig','dest']]

In [133]:
distance_AM = distance_AM.rename(
        columns = dict(zip([i for i in distance_AM.columns if i not in ['orig','dest']],
         [i+ f'_distance_AM' for i in distance_AM.columns if i not in ['orig','dest']])))


In [134]:
distance_MD = distance_MD.rename(
        columns = dict(zip([i for i in distance_MD.columns if i not in ['orig','dest']],
         [i+ f'_distance_MD' for i in distance_MD.columns if i not in ['orig','dest']])))


In [135]:
distance_PM = distance_PM.rename(
        columns = dict(zip([i for i in distance_PM.columns if i not in ['orig','dest']],
         [i+ f'_distance_PM' for i in distance_PM.columns if i not in ['orig','dest']])))


In [136]:
distance_EV = distance_EV.rename(
        columns = dict(zip([i for i in distance_EV.columns if i not in ['orig','dest']],
         [i+ f'_distance_EV' for i in distance_EV.columns if i not in ['orig','dest']])))


In [137]:
cost_EA = cost_EA.rename(
        columns = dict(zip([i for i in cost_EA.columns if i not in ['orig','dest']],
         [i+ f'_cost_EA' for i in cost_EA.columns if i not in ['orig','dest']])))


In [138]:
cost_AM=cost_AM.rename(
        columns = dict(zip([i for i in cost_AM.columns if i not in ['orig','dest']],
         [i+ f'_cost_AM' for i in cost_AM.columns if i not in ['orig','dest']])))


In [139]:
cost_MD=cost_MD.rename(
        columns = dict(zip([i for i in cost_MD.columns if i not in ['orig','dest']],
         [i+ f'_cost_MD' for i in cost_MD.columns if i not in ['orig','dest']])))


In [140]:
cost_PM= cost_PM.rename(
        columns = dict(zip([i for i in cost_PM.columns if i not in ['orig','dest']],
         [i+ f'_cost_PM' for i in cost_PM.columns if i not in ['orig','dest']])))


In [141]:
cost_EV=cost_EV.rename(
        columns = dict(zip([i for i in cost_EV.columns if i not in ['orig','dest']],
         [i+ f'_cost_EV' for i in cost_EV.columns if i not in ['orig','dest']])))


In [142]:
Time_EA=Time_EA.rename(
        columns = dict(zip([i for i in Time_EA.columns if i not in ['orig','dest']],
         [i+ f'_Time_EA' for i in Time_EA.columns if i not in ['orig','dest']])))


In [143]:
Time_AM=Time_AM.rename(
        columns = dict(zip([i for i in Time_AM.columns if i not in ['orig','dest']],
         [i+ f'_Time_AM' for i in Time_AM.columns if i not in ['orig','dest']])))

In [144]:
Time_MD=Time_MD.rename(
        columns = dict(zip([i for i in Time_MD.columns if i not in ['orig','dest']],
         [i+ f'_Time_MD' for i in Time_MD.columns if i not in ['orig','dest']])))

In [145]:
Time_PM=Time_PM.rename(
        columns = dict(zip([i for i in Time_PM.columns if i not in ['orig','dest']],
         [i+ f'_Time_PM' for i in Time_PM.columns if i not in ['orig','dest']])))

In [146]:
Time_EV=Time_EV.rename(
        columns = dict(zip([i for i in Time_EV.columns if i not in ['orig','dest']],
         [i+ f'_Time_EV' for i in Time_EV.columns if i not in ['orig','dest']])))

In [147]:
data_frames = [distance_EA,distance_AM,distance_MD,distance_PM,distance_EV,
              cost_EA,cost_AM,cost_MD,cost_PM,cost_EV,
              Time_EA,Time_AM,Time_MD,Time_PM,Time_EV]
skims = reduce(lambda  left,right: pd.merge(left,right,on=['orig','dest'],
                                            how='outer'), data_frames)

In [148]:
skims.head()

,orig,dest,da_distance_EA,daToll_distance_EA,s2_distance_EA,s2Toll_distance_EA,s3_distance_EA,s3Toll_distance_EA,walk_distance_EA,bike_distance_EA,da_distance_AM,daToll_distance_AM,s2_distance_AM,s2Toll_distance_AM,s3_distance_AM,s3Toll_distance_AM,walk_distance_AM,bike_distance_AM,da_distance_MD,daToll_distance_MD,s2_distance_MD,s2Toll_distance_MD,s3_distance_MD,s3Toll_distance_MD,walk_distance_MD,bike_distance_MD,da_distance_PM,daToll_distance_PM,s2_distance_PM,s2Toll_distance_PM,s3_distance_PM,s3Toll_distance_PM,walk_distance_PM,bike_distance_PM,da_distance_EV,daToll_distance_EV,s2_distance_EV,s2Toll_distance_EV,s3_distance_EV,s3Toll_distance_EV,...,s3_Time_AM,s3Toll_Time_AM,walk_Time_AM,bike_Time_AM,wTrnW_Time_AM,dTrnW_Time_AM,wTrnD_Time_AM,da_Time_MD,daToll_Time_MD,s2_Time_MD,s2Toll_Time_MD,s3_Time_MD,s3Toll_Time_MD,walk_Time_MD,bike_Time_MD,wTrnW_Time_MD,dTrnW_Time_MD,wTrnD_Time_MD,da_Time_PM,daToll_Time_PM,s2_Time_PM,s2Toll_Time_PM,s3_Time_PM,s3Toll_Time_PM,walk_Time_PM,bike_Time_PM,wTrnW_Time_PM,dTrnW_Time_PM,wTrnD_Time_PM,da_Time_EV,daToll_Time_EV,s2_Time_EV,s2Toll_Time_EV,s3_Time_EV,s3Toll_Time_EV,walk_Time_EV,bike_Time_EV,wTrnW_Time_EV,dTrnW_Time_EV,wTrnD_Time_EV
0,1,1,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,0.12,...,5.80,5.80,2.4,0.60,9999.00,9999.00,9999.00,5.80,5.80,5.80,5.80,5.80,5.80,2.4,0.60,9999.00,9999.00,9999.00,5.80,5.80,5.80,5.80,5.80,5.80,2.4,0.60,9999.00,9999.00,9999.00,5.80,5.80,5.80,5.80,5.80,5.80,2.4,0.60,9999.00,9999.00,9999.00
1,1,2,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,0.24,...,6.13,6.13,4.8,1.20,7.20,5.45,3.79,6.14,6.14,6.14,6.14,6.14,6.14,4.8,1.20,8.20,5.90,3.86,6.15,6.15,6.15,6.15,6.15,6.15,4.8,1.20,5.08,3.58,3.68,6.13,6.13,6.13,6.13,6.13,6.13,4.8,1.20,8.67,6.97,4.93
2,1,3,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,0.44,...,6.59,6.59,8.8,2.20,9.70,5.93,4.39,6.59,6.59,6.59,6.59,6.59,6.59,8.8,2.20,10.70,7.62,4.46,6.60,6.60,6.60,6.60,6.60,6.60,8.8,2.20,8.15,5.94,4.28,6.58,6.58,6.58,6.58,6.58,6.58,8.8,2.20,12.70,9.02,5.53
3,1,4,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,0.41,...,6.63,6.63,8.2,2.05,6.10,4.40,3.73,6.65,6.65,6.65,6.65,6.65,6.65,8.2,2.05,6.18,4.28,3.80,6.64,6.64,6.64,6.64,6.64,6.64,8.2,2.05,6.03,4.33,3.62,6.62,6.62,6.62,6.62,6.62,6.62,8.2,2.05,7.10,5.40,5.40
4,1,5,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,0.68,...,7.28,7.28,13.6,3.40,7.42,5.38,4.85,7.45,7.45,7.45,7.45,7.45,7.45,13.6,3.40,7.64,5.53,4.93,7.37,7.37,7.37,7.37,7.37,7.37,13.6,3.40,7.38,5.32,4.78,7.27,7.27,7.27,7.27,7.27,7.27,13.6,3.40,9.11,7.13,5.85


In [109]:
# TOD_list = ['EA','AM','MD','PM','EV']
# var_list = ['Distance','Cost','Time']
# skim_dir = '/home/data/fall_2018/mtc_skims'
# skim_dict = dict()


# for var in var_list:
#     df_base = pd.read_csv(skim_dir + f'/{var}SkimsDatabaseEA.csv').replace(-999, 9999)
# #     df_base['TOD'] = 'EA'
#     df_base.rename(
#         columns = dict(zip([i for i in df_base.columns if i not in ['orig','dest']],
#          [i+ f'_{var}'+ f'_EA' for i in df_base.columns if i not in ['orig','dest']])))
#     for TOD in TOD_list[1:]:
#         df_tod = pd.read_csv(skim_dir + f'/{var}SkimsDatabase{TOD}.csv').replace(-999, 9999)
#         df_tod.rename(
#         columns = dict(zip([i for i in df_tod.columns if i not in ['orig','dest']],
#          [i+ f'_{var}'+ f'_{TOD}' for i in df_tod.columns if i not in ['orig','dest']])))
#         df_base = df_base.merge(df_tod, how = 'left', on = ['orig','dest'])


In [ ]:
# TOD_list = ['EA','AM','MD','PM','EV']
# var_list = ['Distance','Cost','Time']
# skim_dir = '/home/data/fall_2018/mtc_skims'
# skim_dict = dict()


# for var in var_list:
#     df_base = pd.read_csv(skim_dir + f'/{var}SkimsDatabaseEA.csv').replace(-999, 9999)
#     df_base.rename(
#         columns = dict(zip([i for i in df_base.columns],
#          [i+ f'_{var}'+ f'_EA' for i in df_base.columns])))
#     for TOD in TOD_list[1:]:
#         df_tod = pd.read_csv(skim_dir + f'/{var}SkimsDatabase{TOD}.csv').replace(-999, 9999)
#         df_tod.rename(
#         columns = dict(zip([i for i in df_tod.columns if i not in ['orig','dest']],
#          [i+ f'_{var}'+ f'_{TOD}' for i in df_tod.columns if i not in ['orig','dest']])))
#         df_base = df_base.merge(df_tod, how = 'left', on = ['orig','dest'])

In [ ]:
# for TOD in TOD_list:
#         df_base = pd.read_csv(skim_dir + f'/DistanceSkimsDatabase{TOD}.csv').replace(-999, 9999)
#         df_base.rename(
#         columns = dict(zip([i for i in df_base.columns if i not in ['orig','dest']],
#          [i+ f'_{TOD}' for i in df_base.columns if i not in ['orig','dest']])))
#         df_distance = df_base.merge(df_tod, how = 'left', on = ['orig','dest'])

In [110]:
# df_base.head()

,orig,dest,da_x,daToll_x,s2_x,s2Toll_x,s3_x,s3Toll_x,walk_x,bike_x,wTrnW_x,dTrnW_x,wTrnD_x,da_y,daToll_y,s2_y,s2Toll_y,s3_y,s3Toll_y,walk_y,bike_y,wTrnW_y,dTrnW_y,wTrnD_y,da_x,daToll_x,s2_x,s2Toll_x,s3_x,s3Toll_x,walk_x,bike_x,wTrnW_x,dTrnW_x,wTrnD_x,da_y,daToll_y,s2_y,s2Toll_y,s3_y,s3Toll_y,walk_y,bike_y,wTrnW_y,dTrnW_y,wTrnD_y,da,daToll,s2,s2Toll,s3,s3Toll,walk,bike,wTrnW,dTrnW,wTrnD
0,1,1,5.80,5.80,5.80,5.80,5.80,5.80,2.4,0.60,9999.00,9999.00,9999.00,5.80,5.80,5.80,5.80,5.80,5.80,2.4,0.60,9999.00,9999.00,9999.00,5.80,5.80,5.80,5.80,5.80,5.80,2.4,0.60,9999.00,9999.00,9999.00,5.80,5.80,5.80,5.80,5.80,5.80,2.4,0.60,9999.00,9999.00,9999.00,5.80,5.80,5.80,5.80,5.80,5.80,2.4,0.60,9999.00,9999.00,9999.00
1,1,2,6.13,6.13,6.13,6.13,6.13,6.13,4.8,1.20,11.57,8.75,7.98,6.13,6.13,6.13,6.13,6.13,6.13,4.8,1.20,7.20,5.45,3.79,6.14,6.14,6.14,6.14,6.14,6.14,4.8,1.20,8.20,5.90,3.86,6.15,6.15,6.15,6.15,6.15,6.15,4.8,1.20,5.08,3.58,3.68,6.13,6.13,6.13,6.13,6.13,6.13,4.8,1.20,8.67,6.97,4.93
2,1,3,6.58,6.58,6.58,6.58,6.58,6.58,8.8,2.20,13.97,11.60,9.08,6.59,6.59,6.59,6.59,6.59,6.59,8.8,2.20,9.70,5.93,4.39,6.59,6.59,6.59,6.59,6.59,6.59,8.8,2.20,10.70,7.62,4.46,6.60,6.60,6.60,6.60,6.60,6.60,8.8,2.20,8.15,5.94,4.28,6.58,6.58,6.58,6.58,6.58,6.58,8.8,2.20,12.70,9.02,5.53
3,1,4,6.62,6.62,6.62,6.62,6.62,6.62,8.2,2.05,8.17,5.80,8.21,6.63,6.63,6.63,6.63,6.63,6.63,8.2,2.05,6.10,4.40,3.73,6.65,6.65,6.65,6.65,6.65,6.65,8.2,2.05,6.18,4.28,3.80,6.64,6.64,6.64,6.64,6.64,6.64,8.2,2.05,6.03,4.33,3.62,6.62,6.62,6.62,6.62,6.62,6.62,8.2,2.05,7.10,5.40,5.40
4,1,5,7.27,7.27,7.27,7.27,7.27,7.27,13.6,3.40,11.17,8.79,8.66,7.28,7.28,7.28,7.28,7.28,7.28,13.6,3.40,7.42,5.38,4.85,7.45,7.45,7.45,7.45,7.45,7.45,13.6,3.40,7.64,5.53,4.93,7.37,7.37,7.37,7.37,7.37,7.37,13.6,3.40,7.38,5.32,4.78,7.27,7.27,7.27,7.27,7.27,7.27,13.6,3.40,9.11,7.13,5.85


In [306]:
# trips['TOD'] = (
#     ((trips.origin_ET.between(3,6,inclusive = False)) | (trips.origin_ET==3))*1 +
#     ((trips.origin_ET.between(6,9,inclusive = False)) | (trips.origin_ET==6))*2 +
#     ((trips.origin_ET.between(9,15.5,inclusive = False)) | (trips.origin_ET==9))*3 +
#     ((trips.origin_ET.between(15.5,18.5,inclusive = False)) | (trips.origin_ET==15.5))*4 +
#     ((trips.origin_ET>=18.5))*5 +
#     ((trips.origin_ET.between(0,3,inclusive = False)) | (trips.origin_ET==0))*5
# )
# TOD_dict = dict(zip(['EA', 'AM', 'MD', 'PM', 'EV'],range(1,6)))
# trips['TOD'] = trips['TOD'].replace(TOD_dict)

In [307]:
trips = trips1

# for var in var_list: trips[var] = 0
trips.rename(columns = {'Ozone':'orig','Dzone':'dest'},inplace = True)

In [108]:
# skim_dict

{'Distance':          orig  dest     da  daToll     s2  s2Toll     s3  s3Toll   walk  \
 0           1     1   0.12    0.12   0.12    0.12   0.12    0.12   0.12   
 1           1     2   0.24    0.24   0.24    0.24   0.24    0.24   0.24   
 2           1     3   0.44    0.44   0.44    0.44   0.44    0.44   0.44   
 3           1     4   0.41    0.41   0.41    0.41   0.41    0.41   0.41   
 4           1     5   0.68    0.68   0.68    0.68   0.68    0.68   0.68   
 5           1     6   0.97    0.97   0.97    0.97   0.97    0.97   0.97   
 6           1     7   0.98    0.98   0.98    0.98   0.98    0.98   0.98   
 7           1     8   1.15    1.15   1.15    1.15   1.15    1.15   1.15   
 8           1     9   1.57    1.57   1.57    1.57   1.57    1.57   1.56   
 9           1    10   1.59    1.59   1.59    1.59   1.59    1.59   1.59   
 10          1    11   1.15    1.15   1.15    1.15   1.15    1.15   1.13   
 11          1    12   0.66    0.66   0.66    0.66   0.66    0.66   0.66   


In [104]:
# trips1 = trips.copy()
# for var in var_list:
#     trips1 = trips1.merge(skim_dict[var].rename(
#         columns = dict(zip([i for i in skim_dict[var].columns if i not in ['orig','dest','TOD']],
#          [i+ f'_{var}' for i in skim_dict[var].columns if i not in ['orig','dest','TOD']]))                                          
#                                            ), how = 'left',left_on = ['O','D','TOD'], 
#     right_on = ['orig','dest','TOD'])
#     trips1.drop(['orig','dest'],axis = 1,inplace = True)

In [105]:
# trips1.head()

,Unnamed: 0,HHPER,HHPERTRIP,origin,origin_dwell,origin_ST,origin_ET,trip_ET,TT,O,D,Oxcord,Oycord,Dxcord,Dycord,MODE,TOD,Distance,Cost,Time,da_Distance,daToll_Distance,s2_Distance,s2Toll_Distance,s3_Distance,s3Toll_Distance,walk_Distance,bike_Distance,da_Cost,daToll_Cost,s2_Cost,s2Toll_Cost,s3_Cost,s3Toll_Cost,wTrnW_Cost,dTrnW_Cost,wTrnD_Cost,da_Time,daToll_Time,s2_Time,s2Toll_Time,s3_Time,s3Toll_Time,walk_Time,bike_Time,wTrnW_Time,dTrnW_Time,wTrnD_Time
0,0,10352742,1.035274e+09,home,13.583333,19.166667,8.75,9.166667,0.416667,424,700,-122.017274,37.388281,-121.664242,37.151629,drive_alone,AM,0,0,0,31.75,31.75,31.75,31.75,31.75,31.75,30.13,30.13,673.10,673.10,384.63,384.63,269.24,269.24,161.0,171.0,161.0,36.21,36.21,35.73,35.73,35.73,35.73,602.6,150.65,151.29,127.38,126.90
1,1,10352742,1.035274e+09,work,7.583333,9.166667,16.75,19.166667,2.416667,700,424,-121.664242,37.151629,-122.017274,37.388281,drive_alone,PM,0,0,0,31.76,31.76,31.79,31.79,31.79,31.79,30.13,30.13,673.31,673.31,385.11,385.11,269.58,269.58,161.0,161.0,332.0,37.94,37.94,36.75,36.75,36.75,36.75,602.6,150.65,151.81,127.42,123.08
2,2,10414944,1.041494e+09,home,14.500000,18.000000,8.50,8.750000,0.250000,1095,1152,-122.091521,38.010760,-122.118103,37.885783,drive_alone,AM,0,0,0,12.56,12.56,14.13,14.13,14.13,14.13,12.23,12.23,266.27,266.27,171.17,171.17,119.82,119.82,322.0,141.0,161.0,23.61,23.61,22.17,22.17,22.17,22.17,244.6,61.15,104.21,45.15,57.90
3,3,10414944,1.041494e+09,work,8.750000,8.750000,17.50,18.000000,0.500000,1152,1095,-122.118103,37.885783,-122.091521,38.010760,drive_alone,PM,0,0,0,13.82,13.82,13.82,13.82,13.82,13.82,12.23,12.23,292.98,292.98,167.42,167.42,117.19,117.19,141.0,161.0,141.0,22.88,22.88,22.25,22.25,22.25,22.25,244.6,61.15,107.96,60.61,47.95
4,4,10417661,1.041766e+09,home,13.250000,18.750000,8.00,9.000000,1.000000,241,69,-122.414811,37.610585,-122.447100,37.779020,drive_alone,AM,0,0,0,15.04,15.04,15.04,15.04,15.04,15.04,14.61,14.61,318.85,318.85,182.20,182.20,127.54,127.54,563.0,454.0,402.0,28.09,28.09,28.09,28.09,28.09,28.09,292.2,73.05,88.92,63.61,62.11


In [308]:
trips.head()

,Unnamed: 0,HHPER,HHPERTRIP,origin,origin_dwell,origin_ST,origin_ET,trip_ET,TT,orig,dest,Oxcord,Oycord,Dxcord,Dycord,MODE,TOD,SAMPN,PERNO,female,AGE,HISP,white,black,native,asian,PI,immigrant,nolic,JOBS,HOURS,EDUCA,WSCHED,DISAB,HHVEH,HHBIC,rent,INCOM,HHSIZ
0,2,10414944,1.041494e+09,home,14.50,18.00,8.5,8.75,0.25,1095.0,1152.0,-122.091521,38.010760,-122.118103,37.885783,drive_alone,2,1041494,4,1.0,36.0,0,0,0,1,0,0,0.0,0.0,2.0,8.0,2.0,2.0,0.0,3,0.0,0.0,5.0,4
1,3,10414944,1.041494e+09,work,8.75,8.75,17.5,18.00,0.50,1152.0,1095.0,-122.118103,37.885783,-122.091521,38.010760,drive_alone,4,1041494,4,1.0,36.0,0,0,0,1,0,0,0.0,0.0,2.0,8.0,2.0,2.0,0.0,3,0.0,0.0,5.0,4
2,4,10417661,1.041766e+09,home,13.25,18.75,8.0,9.00,1.00,241.0,69.0,-122.414811,37.610585,-122.447100,37.779020,drive_alone,2,1041766,1,1.0,43.0,0,0,0,0,1,0,1.0,0.0,1.0,40.0,6.0,2.0,0.0,2,1.0,0.0,10.0,3
3,5,10417661,1.041766e+09,work,8.50,9.00,17.5,18.75,1.25,69.0,241.0,-122.447100,37.779020,-122.414811,37.610585,drive_alone,4,1041766,1,1.0,43.0,0,0,0,0,1,0,1.0,0.0,1.0,40.0,6.0,2.0,0.0,2,1.0,0.0,10.0,3
4,6,10417662,1.041766e+09,home,13.00,19.00,8.0,8.50,0.50,241.0,282.0,-122.414811,37.610585,-122.328418,37.536798,drive_alone,2,1041766,2,0.0,44.0,0,0,0,0,1,0,1.0,0.0,1.0,60.0,3.0,2.0,0.0,2,1.0,0.0,10.0,3


In [309]:
trips1 = pd.merge(trips, skims, on=['orig','dest'])

In [310]:
trips1.head()

,Unnamed: 0,HHPER,HHPERTRIP,origin,origin_dwell,origin_ST,origin_ET,trip_ET,TT,orig,dest,Oxcord,Oycord,Dxcord,Dycord,MODE,TOD,SAMPN,PERNO,female,AGE,HISP,white,black,native,asian,PI,immigrant,nolic,JOBS,HOURS,EDUCA,WSCHED,DISAB,HHVEH,HHBIC,rent,INCOM,HHSIZ,da_distance_EA,...,s3_Time_AM,s3Toll_Time_AM,walk_Time_AM,bike_Time_AM,wTrnW_Time_AM,dTrnW_Time_AM,wTrnD_Time_AM,da_Time_MD,daToll_Time_MD,s2_Time_MD,s2Toll_Time_MD,s3_Time_MD,s3Toll_Time_MD,walk_Time_MD,bike_Time_MD,wTrnW_Time_MD,dTrnW_Time_MD,wTrnD_Time_MD,da_Time_PM,daToll_Time_PM,s2_Time_PM,s2Toll_Time_PM,s3_Time_PM,s3Toll_Time_PM,walk_Time_PM,bike_Time_PM,wTrnW_Time_PM,dTrnW_Time_PM,wTrnD_Time_PM,da_Time_EV,daToll_Time_EV,s2_Time_EV,s2Toll_Time_EV,s3_Time_EV,s3Toll_Time_EV,walk_Time_EV,bike_Time_EV,wTrnW_Time_EV,dTrnW_Time_EV,wTrnD_Time_EV
0,2,10414944,1.041494e+09,home,14.50,18.00,8.5,8.75,0.25,1095,1152,-122.091521,38.010760,-122.118103,37.885783,drive_alone,2,1041494,4,1.0,36.0,0,0,0,1,0,0,0.0,0.0,2.0,8.0,2.0,2.0,0.0,3,0.0,0.0,5.0,4,14.13,...,22.17,22.17,244.6,61.15,104.21,45.15,57.90,20.16,20.16,20.16,20.16,20.16,20.16,244.6,61.15,107.86,48.21,57.95,21.09,21.09,20.46,20.46,20.46,20.46,244.6,61.15,107.91,47.95,60.56,19.94,19.94,19.94,19.94,19.94,19.94,244.6,61.15,9999.00,50.56,9999.00
1,3,10414944,1.041494e+09,work,8.75,8.75,17.5,18.00,0.50,1152,1095,-122.118103,37.885783,-122.091521,38.010760,drive_alone,4,1041494,4,1.0,36.0,0,0,0,1,0,0,0.0,0.0,2.0,8.0,2.0,2.0,0.0,3,0.0,0.0,5.0,4,13.82,...,19.89,19.89,244.6,61.15,106.63,58.07,46.74,19.86,19.86,19.86,19.86,19.86,19.86,244.6,61.15,107.89,58.07,48.21,22.88,22.88,22.25,22.25,22.25,22.25,244.6,61.15,107.96,60.61,47.95,19.61,19.61,19.61,19.61,19.61,19.61,244.6,61.15,9999.00,9999.00,50.56
2,4,10417661,1.041766e+09,home,13.25,18.75,8.0,9.00,1.00,241,69,-122.414811,37.610585,-122.447100,37.779020,drive_alone,2,1041766,1,1.0,43.0,0,0,0,0,1,0,1.0,0.0,1.0,40.0,6.0,2.0,0.0,2,1.0,0.0,10.0,3,15.04,...,28.09,28.09,292.2,73.05,88.92,63.61,62.11,27.38,27.38,27.38,27.38,27.38,27.38,292.2,73.05,87.59,61.27,63.35,28.71,28.71,28.71,28.71,28.71,28.71,292.2,73.05,86.08,59.77,64.85,26.77,26.77,26.77,26.77,26.77,26.77,292.2,73.05,101.71,67.06,73.69
3,5,10417661,1.041766e+09,work,8.50,9.00,17.5,18.75,1.25,69,241,-122.447100,37.779020,-122.414811,37.610585,drive_alone,4,1041766,1,1.0,43.0,0,0,0,0,1,0,1.0,0.0,1.0,40.0,6.0,2.0,0.0,2,1.0,0.0,10.0,3,15.88,...,25.46,25.46,290.0,72.50,86.78,62.95,60.63,24.94,24.94,24.94,24.94,24.94,24.94,290.0,72.50,88.69,64.27,61.45,25.88,25.88,25.88,25.88,25.88,25.88,290.0,72.50,86.15,65.03,59.66,23.82,23.82,23.82,23.82,23.82,23.82,290.0,72.50,101.41,73.69,66.76
4,6,10417662,1.041766e+09,home,13.00,19.00,8.0,8.50,0.50,241,282,-122.414811,37.610585,-122.328418,37.536798,drive_alone,2,1041766,2,0.0,44.0,0,0,0,0,1,0,1.0,0.0,1.0,60.0,3.0,2.0,0.0,2,1.0,0.0,10.0,3,12.57,...,18.81,18.81,195.2,48.80,76.40,63.99,57.31,18.76,18.76,18.76,18.76,18.76,18.76,195.2,48.80,74.81,62.40,69.03,18.78,18.78,18.78,18.78,18.78,18.78,195.2,48.80,73.63,61.22,67.85,18.68,18.68,18.68,18.68,18.68,18.68,195.2,48.80,99.02,86.61,74.58


In [312]:
trips1.to_csv('/home/emma/ual_model_workspace/fall-2018-models/notebooks-emma/HWtrips_withSkims_102418.csv')

In [313]:
trips2 = trips1.loc[trips1['origin'] == 'home']
trips3 = trips1.loc[trips1['origin'] == 'work']

In [314]:
trips2.TOD.value_counts()

1    3380
2     986
0     436
3      88
4      48
Name: TOD, dtype: int64

In [321]:
trips2.INCOM.value_counts()

7.0     1274
8.0      779
6.0      748
5.0      659
10.0     455
9.0      395
4.0      286
3.0      169
2.0      133
1.0       40
Name: INCOM, dtype: int64

In [322]:
trips2.EDUCA.value_counts()

6.0    1640
5.0    1612
3.0     612
2.0     493
4.0     432
1.0     149
Name: EDUCA, dtype: int64

In [324]:
## change hours to something more interpretable
# trips2.HOURS.value_counts()

In [ ]:
# trips2

# Time_MD=Time_MD.rename(
#         columns = dict(zip([i for i in Time_MD.columns if i not in ['orig','dest']],
#          [i+ f'_Time_MD' for i in Time_MD.columns if i not in ['orig','dest']])))

# tripsP=tripsP.rename(columns = {'place':'origin','ET_num':'origin_ET','WXCORD_lookup':'WXCORD','WYCORD_lookup':'WYCORD'})

In [315]:
@orca.table(cache=True)
def tripsA():
    return trips2

In [316]:
trips.columns

Index(['Unnamed: 0', 'HHPER', 'HHPERTRIP', 'origin', 'origin_dwell',
       'origin_ST', 'origin_ET', 'trip_ET', 'TT', 'orig', 'dest', 'Oxcord',
       'Oycord', 'Dxcord', 'Dycord', 'MODE', 'TOD', 'SAMPN', 'PERNO', 'female',
       'AGE', 'HISP', 'white', 'black', 'native', 'asian', 'PI', 'immigrant',
       'nolic', 'JOBS', 'HOURS', 'EDUCA', 'WSCHED', 'DISAB', 'HHVEH', 'HHBIC',
       'rent', 'INCOM', 'HHSIZ'],
      dtype='object')

In [319]:
#bike and walk variables don't differ by time of day
#need to combine tables 2 and 3 so choosing HW and WH times together
#change age, jobs, hours, educa, hhveh, hhbic, incom, and hhsiz to more interpretable categories
#test origin_dwell, TT?, use just walk distance?
#should any skims data be included here? because time categories are big enough that mode doesn't really influence choice?
#but then why accessibility variables? (total employment within 15 and 30 min by auto and transit for depart and return periods - 
    #and should that be accessibility around home or work?)

m = SmallMultinomialLogitStep()
m.name = 'TOD_choice'
m.tables = ['tripsA']
m.choice_column = 'TOD'
m.model_expression = OrderedDict([
    ('intercept', [0,2,3,4]), 
#     ('da_distance_EA',[0]),
#     ('da_distance_AM',[1]),
#     ('da_distance_MD',[2]),
#     ('da_distance_PM',[3]),
#     ('da_distance_EV',[4]),
    ('da_cost_EA',[0]),
    ('da_cost_AM',[1]),
#     ('da_cost_MD',[2]),
#     ('da_cost_PM',[3]),
#     ('da_cost_EV',[4]),
#     ('da_Time_EA',[0]),
    ('da_Time_AM',[1]),
    ('da_Time_MD',[2]),
    ('da_Time_PM',[3]),
#     ('da_Time_EV',[4]),

#     ('s2_distance_EA',[0,2,3,4]),
#     ('s2_distance_AM',[0,2,3,4]),
#     ('s2_distance_MD',[0,2,3,4]),
#     ('s2_distance_PM',[0,2,3,4]),
#     ('s2_distance_EV',[0,2,3,4]),
#     ('s2_cost_EA',[0,2,3,4]),
#     ('s2_cost_AM',[0,2,3,4]),
#     ('s2_cost_MD',[0,2,3,4]),
#     ('s2_cost_PM',[0,2,3,4]),
#     ('s2_cost_EV',[0,2,3,4]),
#     ('s2_Time_EA',[0,2,3,4]),
#     ('s2_Time_AM',[0,2,3,4]),
#     ('s2_Time_MD',[0,2,3,4]),
#     ('s2_Time_PM',[0,2,3,4]),
#     ('s2_Time_EV',[0,2,3,4]),
    
#     ('bike_distance_EA',[0,2,3,4]),
# #     ('bike_distance_AM',[0,2,3,4]),
# #     ('bike_distance_MD',[0,2,3,4]),
# #     ('bike_distance_PM',[0,2,3,4]),
# #     ('bike_distance_EV',[0,2,3,4]),
# #     ('bike_cost_EA',[0,1,2,3]),
# #     ('bike_cost_AM',[0,1,2,3]),
# #     ('bike_cost_MD',[0,1,2,3]),
# #     ('bike_cost_PM',[0,1,2,3]),
# #     ('bike_cost_EV',[0,1,2,3]),
#     ('bike_Time_EA',[0,2,3,4]),
# #     ('bike_Time_AM',[0,2,3,4]),
# #     ('bike_Time_MD',[0,2,3,4]),
# #     ('bike_Time_PM',[0,2,3,4]),
# #     ('bike_Time_EV',[0,2,3,4]),
    
# #     ('wTrnW_distance_EA',[0,1,2,3]),
# #     ('wTrnW_distance_AM',[0,1,2,3]),
# #     ('wTrnW_distance_MD',[0,1,2,3]),
# #     ('wTrnW_distance_PM',[0,1,2,3]),
# #     ('wTrnW_distance_EV',[0,1,2,3]),
#     ('wTrnW_cost_EA',[0]),
#     ('wTrnW_cost_AM',[1]),
#     ('wTrnW_cost_MD',[2]),
#     ('wTrnW_cost_PM',[3]),
#     ('wTrnW_cost_EV',[4]),
#     ('wTrnW_Time_EA',[0]),
#     ('wTrnW_Time_AM',[1]),
#     ('wTrnW_Time_MD',[2]),
#     ('wTrnW_Time_PM',[3]),
#     ('wTrnW_Time_EV',[4]),
    
# #     ('wTrnD_distance_EA',[0,1,2,3]),
# #     ('wTrnD_distance_AM',[0,1,2,3]),
# #     ('wTrnD_distance_MD',[0,1,2,3]),
# #     ('wTrnD_distance_PM',[0,1,2,3]),
# #     ('wTrnD_distance_EV',[0,1,2,3]),
#     ('wTrnD_cost_EA',[0]),
#     ('wTrnD_cost_AM',[1]),
#     ('wTrnD_cost_MD',[2]),
#     ('wTrnD_cost_PM',[3]),
#     ('wTrnD_cost_EV',[4]),
#     ('wTrnD_Time_EA',[0]),
#     ('wTrnD_Time_AM',[1]),
#     ('wTrnD_Time_MD',[2]),
#     ('wTrnD_Time_PM',[3]),
#     ('wTrnD_Time_EV',[4]),
    
# #     ('dTrnW_distance_EA',[0,1,2,3]),
# #     ('dTrnW_distance_AM',[0,1,2,3]),
# #     ('dTrnW_distance_MD',[0,1,2,3]),
# #     ('dTrnW_distance_PM',[0,1,2,3]),
# #     ('dTrnW_distance_EV',[0,1,2,3]),
#     ('dTrnW_cost_EA',[0]),
#     ('dTrnW_cost_AM',[1]),
#     ('dTrnW_cost_MD',[2]),
    ('dTrnW_cost_PM',[3]),
#     ('dTrnW_cost_EV',[4]),
#     ('dTrnW_Time_EA',[0]),
#     ('dTrnW_Time_AM',[1]),
#     ('dTrnW_Time_MD',[2]),
    ('dTrnW_Time_PM',[3]),
#     ('dTrnW_Time_EV',[4]),
    
#     ('walk_distance_EA',[0,2,3,4]),
# #     ('walk_distance_AM',[1]),
# #     ('walk_distance_MD',[2]),
# #     ('walk_distance_PM',[3]),
# #     ('walk_distance_EV',[4]),
# #     ('walk_cost_EA',[0,1,2,3]),
# #     ('walk_cost_AM',[0,1,2,3]),
# #     ('walk_cost_MD',[0,1,2,3]),
# #     ('walk_cost_PM',[0,1,2,3]),
# #     ('walk_cost_EV',[0,1,2,3]),
#     ('walk_Time_EA',[0,2,3,4]),
# #     ('walk_Time_AM',[1]),
# #     ('walk_Time_MD',[2]),
# #     ('walk_Time_PM',[3]),
# #     ('walk_Time_EV',[4]),
    
    ('AGE',[0,2,3,4]),
#     ('HISP',[0,2,3,4]),
#     ('black',[0,2,3,4]),
#     ('native',[0,2,3,4]),
#     ('asian',[0,2,3,4]),
#     ('PI',[0,2,3,4]),
    ('INCOM',[0,2,3,4]),
    ('EDUCA',[0,2,3,4]),
    ('immigrant',[0,2,3,4]),
    ('nolic',[0,2,3,4]),
    ('JOBS',[0,2,3,4]),
    ('HOURS',[0,2,3,4]),
    ('WSCHED',[0,2,3,4]),
#     ('DISAB',[0,2,3,4]),
#     ('HHVEH',[0,2,3,4]),
#     ('HHBIC',[0,2,3,4]),
#     ('rent',[0,2,3,4]),
    ('HHSIZ',[0,2,3,4]),
    
])

In [320]:
m.fit()

Log-likelihood at zero: -7,947.4044
Initial Log-likelihood: -7,947.4044
Estimation Time for Point Estimation: 1.88 seconds.
Final log-likelihood: -4,058.3423
                     Multinomial Logit Model Regression Results                    
Dep. Variable:                     _chosen   No. Observations:                4,938
Model:             Multinomial Logit Model   Df Residuals:                    4,887
Method:                                MLE   Df Model:                           51
Date:                     Wed, 24 Oct 2018   Pseudo R-squ.:                   0.489
Time:                             20:23:51   Pseudo R-bar-squ.:               0.483
AIC:                             8,218.685   Log-Likelihood:             -4,058.342
BIC:                             8,550.425   LL-Null:                    -7,947.404
                      coef    std err          z      P>|z|      [0.025      0.975]
-----------------------------------------------------------------------------------
in